Loan Approval Prediction with Logistic Regression

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Load the dataset
data = pd.read_csv('loan_data.csv')
data.head()

,customer_id,customer_age,customer_income,home_ownership,employment_duration,loan_intent,loan_grade,loan_amnt,loan_int_rate,term_years,historical_default,cred_hist_length,Current_loan_status
0,1.0,22,59000,RENT,123.0,PERSONAL,C,"£35,000.00",16.02,10,Y,3,DEFAULT
1,2.0,21,9600,OWN,5.0,EDUCATION,A,"£1,000.00",11.14,1,NaN,2,NO DEFAULT
2,3.0,25,9600,MORTGAGE,1.0,MEDICAL,B,"£5,500.00",12.87,5,N,3,DEFAULT
3,4.0,23,65500,RENT,4.0,MEDICAL,B,"£35,000.00",15.23,10,N,2,DEFAULT
4,5.0,24,54400,RENT,8.0,MEDICAL,B,"£35,000.00",14.27,10,Y,4,DEFAULT


In [3]:
data.describe()

,customer_id,customer_age,employment_duration,loan_int_rate,term_years,cred_hist_length
count,32583.000000,32586.000000,31691.000000,29470.000000,32586.000000,32586.000000
mean,16289.497806,27.732769,4.790161,11.011553,4.761738,5.804026
std,9405.919628,6.360528,4.142746,3.240440,2.471107,4.055078
min,1.000000,3.000000,0.000000,5.420000,1.000000,2.000000
25%,8144.500000,23.000000,2.000000,7.900000,3.000000,3.000000
50%,16288.000000,26.000000,4.000000,10.990000,4.000000,4.000000
75%,24433.500000,30.000000,7.000000,13.470000,7.000000,8.000000
max,32581.000000,144.000000,123.000000,23.220000,10.000000,30.000000


Data Preparation and Preprocessing

In [4]:
print("Column name and no. of non-missing values")
data.count().sort_values(ascending=False)

Column name and no. of non-missing values


customer_age           32586
home_ownership         32586
customer_income        32586
loan_grade             32586
loan_intent            32586
cred_hist_length       32586
term_years             32586
loan_amnt              32585
customer_id            32583
Current_loan_status    32582
employment_duration    31691
loan_int_rate          29470
historical_default     11849
dtype: int64

In [5]:
print("Column name and no. of missing values")
data.isnull().sum()

Column name and no. of missing values


customer_id                3
customer_age               0
customer_income            0
home_ownership             0
employment_duration      895
loan_intent                0
loan_grade                 0
loan_amnt                  1
loan_int_rate           3116
term_years                 0
historical_default     20737
cred_hist_length           0
Current_loan_status        4
dtype: int64

In [6]:
df = data.copy()

Drop what should NOT be features (important note)

In [7]:
df = df.drop(columns=['customer_id'])
df.count()

customer_age           32586
customer_income        32586
home_ownership         32586
employment_duration    31691
loan_intent            32586
loan_grade             32586
loan_amnt              32585
loan_int_rate          29470
term_years             32586
historical_default     11849
cred_hist_length       32586
Current_loan_status    32582
dtype: int64

In [8]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 32586 entries, 0 to 32585
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   customer_age         32586 non-null  int64  
 1   customer_income      32586 non-null  str    
 2   home_ownership       32586 non-null  str    
 3   employment_duration  31691 non-null  float64
 4   loan_intent          32586 non-null  str    
 5   loan_grade           32586 non-null  str    
 6   loan_amnt            32585 non-null  str    
 7   loan_int_rate        29470 non-null  float64
 8   term_years           32586 non-null  int64  
 9   historical_default   11849 non-null  str    
 10  cred_hist_length     32586 non-null  int64  
 11  Current_loan_status  32582 non-null  str    
dtypes: float64(2), int64(3), str(7)
memory usage: 3.0 MB


"loan_amnt" is string, but logically numeric → handled separately (see below)

In [9]:
df["loan_amnt"] = pd.to_numeric(df["loan_amnt"], errors="coerce")

In [10]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 32586 entries, 0 to 32585
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   customer_age         32586 non-null  int64  
 1   customer_income      32586 non-null  str    
 2   home_ownership       32586 non-null  str    
 3   employment_duration  31691 non-null  float64
 4   loan_intent          32586 non-null  str    
 5   loan_grade           32586 non-null  str    
 6   loan_amnt            0 non-null      float64
 7   loan_int_rate        29470 non-null  float64
 8   term_years           32586 non-null  int64  
 9   historical_default   11849 non-null  str    
 10  cred_hist_length     32586 non-null  int64  
 11  Current_loan_status  32582 non-null  str    
dtypes: float64(3), int64(3), str(6)
memory usage: 3.0 MB


Select categorical columns

In [11]:
target = "Current_loan_status"
cat_cols = df.select_dtypes(include=["string", "object"]).columns.tolist()
cat_cols = [c for c in cat_cols if c != target]   # exclude target from features

Select numerical column

In [12]:
num_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()

https://www.kaggle.com/code/alexisbcook/pipelines (Fill null values)

In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [14]:
# Preprocessing for numerical data
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value=0))
])

# Preprocessing for categorical data
cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value="Unknown")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

In [15]:
#combine
preprocessor = ColumnTransformer([
    ("num", num_pipeline, num_cols),
    ("cat", cat_pipeline, cat_cols)
])


In [16]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

my_model = LogisticRegression(max_iter=1000)

model = Pipeline([
    ("preprocess", preprocessor),
    ("model", my_model)
])


In [17]:
from sklearn.model_selection import train_test_split
target = "Current_loan_status"
X = df.drop(columns=[target])
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)
model.fit(X_train, y_train)

/usr/local/python/3.12.1/lib/python3.12/site-packages/sklearn/impute/_base.py:641: UserWarning: Skipping features without any observed values: ['loan_amnt']. At least one non-missing value is needed for imputation with strategy='constant'.
  warnings.warn(


ValueError: Input contains NaN

That error means NaNs are still getting into the model somewhere

In [19]:
all_nan_num = [c for c in num_cols if df[c].isna().all()]
print("All-NaN numeric columns:", all_nan_num)

All-NaN numeric columns: ['loan_amnt']


Because loan_amnt is NaN, median imputer can’t compute so preprocessing and the model crashes.

Drop loan_amnt from features

In [20]:
num_cols = [c for c in num_cols if c != "loan_amnt"]
df = df.drop(columns=["loan_amnt"])

rebuild the preprocessing objects after removing loan_amnt.

In [21]:
target = "Current_loan_status"

# Drop missing targets (recommended)
df = df.dropna(subset=[target]).copy()

# Drop loan_amnt safely
df = df.drop(columns=["loan_amnt"], errors="ignore")

# Recompute column lists (fresh)
cat_cols = df.select_dtypes(include=["object", "string"]).columns.tolist()
cat_cols = [c for c in cat_cols if c != target]

num_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()

# Pipelines
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median"))
])

cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value="Unknown")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

# Preprocessor (rebuild!)
preprocessor = ColumnTransformer([
    ("num", num_pipeline, num_cols),
    ("cat", cat_pipeline, cat_cols)
])

# Model pipeline (rebuild!)
new_model = Pipeline([
    ("preprocess", preprocessor),
    ("model", LogisticRegression(max_iter=1000))
])

In [22]:
X = df.drop(columns=[target])
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)
new_model.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers cont

it worked now, let's make predicions now

In [23]:
predictions = new_model.predict(X_test)

Evaluate the model

In [24]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Accuracy:", accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

Accuracy: 0.9435323001381003
[[1172  195]
 [ 173 4977]]
              precision    recall  f1-score   support

     DEFAULT       0.87      0.86      0.86      1367
  NO DEFAULT       0.96      0.97      0.96      5150

    accuracy                           0.94      6517
   macro avg       0.92      0.91      0.91      6517
weighted avg       0.94      0.94      0.94      6517

